In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [21]:
df = pd.read_csv("precise_learning_dataset.csv")


In [22]:
df

,user_id,academic_year,branch,skills,quiz_q1_response,quiz_q2_response,quiz_q3_response,quiz_q4_response,quiz_q5_response,quiz_q6_response,...,quiz_q8_response,quiz_q9_response,quiz_q10_response,visual_percent,auditory_percent,kinesthetic_percent,recommended_format,recommended_courses,recommended_books,recommended_blogs
0,100000,First Year (FE),Information Technology,"Python, CSS, Docker, AWS",A,A,A,A,A,A,...,A,A,A,100,0,0,Videos,"HTML & CSS Visual Design, SQL Mastery",CSS Secrets,visualizepython.org
1,100001,Third Year (TE),Artificial Intelligence and Machine Learning,"JavaScript, SQL, AWS, React, Java",A,A,A,A,A,A,...,A,A,B,90,10,0,Videos,"HTML & CSS Visual Design, SQL Mastery",Designing Interfaces,frontenddesign.blog
2,100002,Third Year (TE),Electronics & Communication,"C++, SQL, CSS, React",A,A,A,A,A,A,...,A,A,C,90,0,10,Videos,"AWS Cloud Diagrams, SQL Mastery",Designing Interfaces,frontenddesign.blog
3,100003,Second Year (SE),Electronics and Computer Engineering,"Docker, Java, Figma, JavaScript",A,A,A,A,A,A,...,A,B,A,90,10,0,Videos,"SQL Mastery, HTML & CSS Visual Design",Designing Interfaces,frontenddesign.blog
4,100004,Final Year (BE),Electronics & Communication,"JavaScript, AWS, HTML, Figma",A,A,A,A,A,A,...,A,B,B,80,20,0,Videos,"SQL Mastery, HTML & CSS Visual Design",Designing Interfaces,frontenddesign.blog
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59044,159044,Third Year (TE),Information Technology,"CSS, HTML, Java, Docker, SQL",C,C,C,C,C,C,...,C,B,B,0,20,80,Projects,"Docker Workshop, Python Projects Bootcamp",Learn Python by Building,devdojo.com
59045,159045,First Year (FE),Electronics & Communication,"React, CSS, C++, HTML",C,C,C,C,C,C,...,C,B,C,0,10,90,Projects,"Linux Hands-On Lab, Python Projects Bootcamp",Learn Python by Building,devdojo.com
59046,159046,Second Year (SE),Electronics and Computer Engineering,"C++, Java, Figma, JavaScript, Docker, HTML",C,C,C,C,C,C,...,C,C,A,10,0,90,Projects,"Python Projects Bootcamp, Docker Workshop",Learn Python by Building,realpython.com
59047,159047,Final Year (BE),Information Technology,"JavaScript, Java, Figma",C,C,C,C,C,C,...,C,C,B,0,10,90,Projects,"Linux Hands-On Lab, Python Projects Bootcamp",Learn Python by Building,devdojo.com


In [23]:
df = df.drop(columns=["user_id", "recommended_format", 
                      "recommended_courses", "recommended_books", "recommended_blogs"])


In [24]:
# Define target columns
y = df[["visual_percent", "auditory_percent", "kinesthetic_percent"]]

# Preprocess categorical columns
df = df.drop(columns=["visual_percent", "auditory_percent", "kinesthetic_percent"])
categorical_cols = ["academic_year", "branch"] + [f"quiz_q{i}_response" for i in range(1, 11)]
df_encoded = pd.get_dummies(df[categorical_cols])

# Preprocess skills as multi-hot
skill_list = ["Python", "Java", "C++", "SQL", "HTML", "CSS", "JavaScript", "Docker",
              "AWS", "Figma", "React", "Linux"]

for skill in skill_list:
    df_encoded[f"skill_{skill}"] = df["skills"].apply(lambda x: 1 if skill in x else 0)

# Final features and targets
X = df_encoded
y = y

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Optional: Normalize to 100
import numpy as np
y_pred = np.clip(y_pred, 0, None)  # No negatives
y_pred = y_pred / y_pred.sum(axis=1, keepdims=True) * 100

# Evaluate
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae:.2f}")

Mean Absolute Error: 0.01


In [26]:
import joblib

# Save the trained model
joblib.dump(model, 'model.pkl')

# Save the feature column names (important for input formatting during prediction)
joblib.dump(X.columns.tolist(), 'feature_columns.pkl')


['feature_columns.pkl']